МНОГОМЕРНАЯ ОПТИМИЗАЦИЯ НЕПРЕРЫВНЫХ ФУНКЦИЙ

In [18]:
import numpy as np

Целью данного алгоритма является нахождение оптимального решения задачи линейного программирования. Мы сталкиваемся с задачей оптимизации, где требуется максимизировать или минимизировать линейную целевую функцию при соблюдении линейных ограничений. 

In [19]:
c = [1, 1, 0, 0, 0]
A = [[-1, 1, 1, 0, 0],
    [ 1, 0, 0, 1, 0],
    [ 0, 1, 0, 0, 1]]
b = [2, 4, 4]

In [20]:
# Преобразуем данные в единую матрицу
# Данный метод преобразует исходные данные в единую матрицу, 
# представляющую собой расширенную форму задачи линейного программирования.
def to_table(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]

In [21]:
# Проверим, можем ли найти базисное решение
# Этот метод проверяет, можно ли улучшить текущее базисное решение, исходя из последней строки таблицы.
def can_be_improved(table):
    z = table[-1]
    return any(x > 0 for x in z[:-1])

In [22]:
# Найдем границу базисного решения
# Метод определяет позицию опорного элемента (pivot) в таблице, что является ключевым шагом симплекс-метода.
def get_pivot_position(table):
    z = table[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in table[:-1]:
        el = eq[column]
        restrictions.append(float('inf') if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column

In [23]:
# Выполним шаг, вычислив новое базисное решение
# Выполняет шаг симплекс-метода, осуществляя "поворот" к новому базисному решению.
def pivot_step(tableau, pivot_position):
    new_tableau = [[] for _ in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i in range(len(tableau)):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
   
    return new_tableau

In [24]:
# Проверяет, является ли столбец базисным, что важно для определения оптимальности решения.
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

# Извлекает решение из конечной таблицы симплекс-метода.
def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
        
    return solutions

In [25]:
# Запускает алгоритм симплекс-метода с заданными коэффициентами и ограничениями.
def simplex(c, A, b):
    table = to_table(c, A, b)

    while can_be_improved(table):
        pivot_position = get_pivot_position(table)
        table = pivot_step(table, pivot_position)

    return get_solution(table)

In [26]:
# алгоритм завершает свою работу, когда текущее решение становится оптимальным, и результат выводится в переменной solution
solution = simplex(c, A, b)
solution

[4.0, 4.0, 2.0, 0, 0, 0]